In [1]:
import pandas as pd
import numpy as np
from herramientas import obtener_resumen_valores_unicos
import plotly.express as px
import gc
from sklearn.model_selection import train_test_split

In [2]:
#train = pd.read_csv('bbva-data-challenge-2023/archive/universe_train.csv')
#balances = pd.read_csv('bbva-data-challenge-2023/archive/balances.csv')
customers = pd.read_csv('bbva-data-challenge-2023/archive/customers.csv')
liabiities = pd.read_csv('bbva-data-challenge-2023/archive/liabilities.csv')
#movements = pd.read_csv('bbva-data-challenge-2023/archive/movements.csv')
#test = pd.read_csv('bbva-data-challenge-2023/archive/universe_test.csv')
digital = pd.read_csv('bbva-data-challenge-2023/digital.csv')

In [3]:
df = pd.merge(customers, liabiities, on='ID', how='left')

In [4]:
# Reemplazar 'yes' con 1 y 'no' con 0 en ambas columnas
df['product_1_x'] = df['product_1_x'].replace({'Yes': 1, 'No': 0})
df['product_2_x'] = df['product_2_x'].replace({'Yes': 1, 'No': 0})
df['product_3'] = df['product_3'].replace({'Yes': 1, 'No': 0})
df['product_4'] = df['product_4'].replace({'Yes': 1, 'No': 0})
df['ofert_1'] = df['ofert_1'].replace({'Yes': 1, 'No': 0})
df['ofert_2'] = df['ofert_2'].replace({'Yes': 1, 'No': 0})
df['ofert_3'] = df['ofert_3'].replace({'Yes': 1, 'No': 0})

In [5]:
# Primero, crea variables dummy para columna1
df = pd.get_dummies(df, columns=['type_job'], prefix=['type_job'], drop_first=True)

# Luego, crea variables dummy para columna2
df = pd.get_dummies(df, columns=['bureau_risk'], prefix=['bureau_risk'], drop_first=True)

In [6]:
# Crear un DataFrame auxiliar con todos los pares posibles de ID y month
id_values = df['ID'].unique()
months = np.arange(1, 13)
pairs = [(id_val, month) for id_val in id_values for month in months]
aux_df = pd.DataFrame(pairs, columns=['ID', 'month'])

# Fusionar el DataFrame auxiliar con el DataFrame original para rellenar los valores faltantes
df = pd.merge(df, aux_df, on=['ID', 'month'], how='left')

# Ahora df contendrá todas las combinaciones de ID y month posibles, con valores faltantes rellenados con NaN
# Puedes reemplazar los NaN con 1 o cualquier otro valor predeterminado si lo deseas
df['month'].fillna(1, inplace=True)  # Reemplazar NaN con 1

df = df.drop(columns=['period'])

# Primero, crea una columna llamada "set" que indique "train" si product_1_y no es nulo y "test" si lo es
df['set'] = df['product_1_y'].apply(lambda x: 'train' if pd.notna(x) else 'test')

# Ahora, puedes dividir el DataFrame en train y test utilizando la columna "set"
train_df = df[df['set'] == 'train'].drop(columns=['set'])  # Elimina la columna "set" después de dividir
test_df = df[df['set'] == 'test'].drop(columns=['set'])

# Definir las variables predictoras (X)
x  = train_df.drop(columns=['product_1_y'])  # Elimina la columna 'product_1_y' del DataFrame

# Definir la variable objetivo (y)
y = train_df['product_1_y']

X = x.drop(columns=['ID'])

In [7]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un objeto XGBoost Regressor
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio (MSE) en el conjunto de prueba
mse = mean_squared_error(y_test, y_pred)
print(f"Error Cuadrático Medio (MSE): {mse}")

# Puedes ajustar los hiperparámetros y optimizar el modelo según sea necesario

/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

Error Cuadrático Medio (MSE): 0.9304123074497282


In [8]:
x1  = test_df.drop(columns=['product_1_y', 'ID']) 
y_pred_1 = model.predict(x1)

/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

In [9]:
test_df = test_df.drop(columns=['product_1_y']) 
test_df = test_df.reset_index(drop=True)
y_pred_1 = pd.DataFrame(y_pred_1, columns=['product_1_y'])  
X_test = pd.concat([test_df, y_pred_1], axis=1)
X_train = pd.concat([train_df, X_test], axis=0)

In [10]:
obtener_resumen_valores_unicos(X_train)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos
0,ID,object,809579,8895559,0
1,age,float64,15395,8895559,0
2,income,float64,14072,8895559,0
3,time_from_specialized,float64,20324,8895559,0
4,payroll,float64,22474,8895559,0
5,product_1_x,int64,2,8895559,0
6,product_2_x,int64,2,8895559,0
7,product_3,int64,2,8895559,0
8,product_4,int64,2,8895559,0
9,ofert_1,int64,2,8895559,0


# produc_2

In [11]:
# Primero, crea una columna llamada "set" que indique "train" si product_1_y no es nulo y "test" si lo es
X_train['set'] = X_train['product_2_y'].apply(lambda x: 'train' if pd.notna(x) else 'test')

# Ahora, puedes dividir el DataFrame en train y test utilizando la columna "set"
train_df = X_train[X_train['set'] == 'train'].drop(columns=['set'])  # Elimina la columna "set" después de dividir
test_df = X_train[X_train['set'] == 'test'].drop(columns=['set'])

# train_df contendrá las filas con valores no nulos en "product_1_y"
# test_df contendrá las filas con valores nulos en "product_1_y"

In [12]:
# Definir las variables predictoras (X)
x  = train_df.drop(columns=['product_2_y'])  # Elimina la columna 'product_2_y' del DataFrame

# Definir la variable objetivo (y)
y = train_df['product_2_y']

In [13]:
X = x.drop(columns=['ID'])

In [14]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

# Crear un objeto XGBoost Regressor
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio (MSE) en el conjunto de prueba
mse = mean_squared_error(y_test, y_pred)
print(f"Error Cuadrático Medio (MSE): {mse}")

# Puedes ajustar los hiperparámetros y optimizar el modelo según sea necesario

/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

Error Cuadrático Medio (MSE): 1.4000575420291406


In [15]:
x1  = test_df.drop(columns=['product_2_y', 'ID']) 
y_pred_1 = model.predict(x1)

/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

In [16]:
test_df_1 = test_df.drop(columns=['product_2_y']) 
test_df_1 = test_df_1.reset_index(drop=True)
y_pred_1 = pd.DataFrame(y_pred_1, columns=['product_2_y'])  
X_test = pd.concat([test_df_1, y_pred_1], axis=1)
X_train_1 = pd.concat([train_df, X_test], axis=0)

In [17]:
obtener_resumen_valores_unicos(X_train_1)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos
0,ID,object,809579,8895559,0
1,age,float64,15395,8895559,0
2,income,float64,14072,8895559,0
3,time_from_specialized,float64,20324,8895559,0
4,payroll,float64,22474,8895559,0
5,product_1_x,int64,2,8895559,0
6,product_2_x,int64,2,8895559,0
7,product_3,int64,2,8895559,0
8,product_4,int64,2,8895559,0
9,ofert_1,int64,2,8895559,0


In [18]:
del aux_df
del customers
del df
del id_values
del liabiities
del model
del months
del mse
del pairs
del test_df
del test_df_1
del train_df
del X
del x
del X_test

In [19]:
del X_train
del x1
del y
del y_pred
del y_pred_1
del y_test
del y_train

In [20]:
gc.collect()

200

In [21]:
chunk_size = 1000000  # Define el tamaño del fragmento
chunks = [X_train_1[i:i+chunk_size] for i in range(0, len(X_train_1), chunk_size)]

df_merged = pd.DataFrame()  # DataFrame vacío para concatenar los resultados

for chunk in chunks:
    partial_merged = pd.merge(chunk, digital, on='ID', how='inner')
    df_merged = pd.concat([df_merged, partial_merged], ignore_index=True)


: 

In [ ]:
obtener_resumen_valores_unicos(df_merged)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos
0,ID,object,528728,42698676,0
1,age,float64,15233,42698676,0
2,income,float64,13375,42698676,0
3,time_from_specialized,float64,19975,42698676,0
4,payroll,float64,20857,42698676,0
5,product_1_x,int64,2,42698676,0
6,product_2_x,int64,2,42698676,0
7,product_3,int64,2,42698676,0
8,product_4,int64,2,42698676,0
9,ofert_1,int64,2,42698676,0


: 